In [ ]:
#neccessary libraries for colab notebook to work
!pip install swig
!pip install gymnasium
!pip install gymnasium[box2d]

In [ ]:
# @title Neccessary imports
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
from DQN import * 
from DDQN import * 
from DDPG import * 
from PPO import * 
from A2C import * 
from DuelingDDQN import * 
from DuelingDQN import * 
from rule_extraction import *

In [ ]:
# @title Create Environment
# select discrete or continuous environment (continuous only for ddpg)
env_continuous = False
#create environment
env = gym.make("LunarLander-v3", continuous = env_continuous)
#state size , action size
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space)

#select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @title Train Agent

# pick agent to train

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = False)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = True, batch_norm = False)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = True)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

agent = DoubleDQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [256,128],random_seed = 0, per = False, batch_norm = False)
scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

#agent = A2CAgent(state_size = 8, action_size = 4, device = device)
#scores = agent.train_agent(env, num_episodes = 2000, max_t = 500)

#agent = PPOAgent(state_size = 8, action_size = 4, device = device)
#scores = agent.train_agent(env, num_episodes = 2000, max_t = 600)

#agent = DDPGAgent(state_size = 8, action_size = 2, device = device, random_seed=0)
#scores = agent.train_agent(env, n_episodes = 2000, max_t = 500)

In [ ]:
# @title Plot results

#find running mean
scores_window = 10
running_mean = np.convolve(scores, np.ones(scores_window)/scores_window, mode = "valid")
#plot convergence curve and running mean
plt.plot(np.arange(len(scores)), scores)
plt.plot(np.arange(scores_window-1, len(scores)), running_mean, label='10-game Running Mean', color='red')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.grid()
plt.savefig("ppo_cont")
plt.show()

In [ ]:
# @title Watch a Smart Agent

# load the trained agents from the trained_agents folder provided to the google colab terminal and watch the agent solve the environment

# pick if the environment is discrete or continuous
env_cont = False

'''if env_cont = False pick one of the following agents'''

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = False)
#agent.load("checkpoint_dqn.pth")

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = True, batch_norm = False)
#agent.load("checkpoint_dqn_per.pth")

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = True)
#agent.load("checkpoint_dqn_bn.pth")

agent = DoubleDQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [256,128],random_seed = 0, per = False, batch_norm = False)
agent.load("checkpoint_ddqn.pth")

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#agent.load("checkpoint_dueling_dqn.pth")

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#agent.load("checkpoint_dueling_dqn_per.pth")

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#agent.load("checkpoint_dueling_ddqn.pth")

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#agent.load("checkpoint_dueling_ddqn_per.pth")

'''if env_cont = True pick the DDPG'''

#agent = DDPGAgent(state_size = 8, action_size = 2, device = device, random_seed=0)
#agent.load("checkpoint_ddpg_actor.pth", "checkpoint_ddpg_critic.pth")


test_env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = env_cont)
frames = []

state = test_env.reset()[0]
for j in range(600):
    frame = test_env.render()
    frames.append(frame)

    action = agent.act(state)
    next_state, reward, terminated, truncated , _= test_env.step(action)
    done = terminated or truncated
    state = next_state
    if done:
        break

test_env.close()


fig = plt.figure()
plt.axis('off')
ims = [[plt.imshow(frame, animated=True)] for frame in frames]
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
plt.close()

HTML(ani.to_jshtml())


In [174]:
# @title Load Agent
#load agent

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = False)
#agent.load("checkpoint_dqn.pth")

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = True, batch_norm = False)
#agent.load("checkpoint_dqn_per.pth")

#agent = DQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [512,256],random_seed = 0, per = False, batch_norm = True)
#agent.load("checkpoint_dqn_bn.pth")

#agent = DoubleDQNAgent(state_size = 8 , action_size = 4, device = device, hidden_sizes = [256,128],random_seed = 0, per = False, batch_norm = False)
#agent.load("checkpoint_ddqn.pth")

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#agent.load("checkpoint_dueling_dqn.pth")

#agent = DuelingDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#agent.load("checkpoint_dueling_dqn_per.pth")

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = False)
#agent.load("checkpoint_dueling_ddqn.pth")

#agent = DuelingDoubleDQNAgent(state_size = 8 , action_size = 4, device = device,random_seed = 0, per = True)
#agent.load("checkpoint_dueling_ddqn.pth")


In [ ]:
# @title Test Classifier
tree_classifier = get_rules(env , agent, num_episodes = 200, max_depth = 4)
mean = test_classifier(env , tree_classifier , num_of_episodes = 200)
print(f"Classifier : Mean score = {mean}")

In [ ]:
# @title Print Decision Tree
print_rules(tree_classifier)